In [1]:
from numba import float32, complex64, guvectorize, jit, autojit, cuda
import numpy as np
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [ ]:
#@autojit
def lst_grid(lsts, data, wgts=None, lstbins=6300, wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))):
    lstgrid = np.linspace(0, 2*np.pi, lstbins)
    lstres = lstgrid[1]-lstgrid[0]
    if wgts is None: wgts = np.where(np.abs(data) == 0, 0, 1.)
    sumgrid,wgtgrid = 0, 0
    for lst,d,w in zip(lsts,data,wgts):
        dt = lstgrid - lst
        wf = wgtfunc(dt,lstres); wf.shape = (-1,) + (1,)*(data.ndim-1)
        d.shape, w.shape = (1,-1), (1,-1)
        #print lst, dt
        wgtgrid += w * wf
        sumgrid += d * w * wf 
    datgrid = np.where(wgtgrid > 1e-10, sumgrid/wgtgrid, 0)
    #print('real', datgrid)
    return lstgrid, datgrid, wgtgrid
@guvectorize([(float32[:], float32[:,:],float32[:,:], float32[:,:])], '(n),(n,m)->(n,m),(n,m)')
def g_cpu(lsts, data, lstsg, datag):
    lstsg, datag, _ = lst_grid(lsts, data)
    #lstsg, datag = lsts, data

In [ ]:
def glst_grid(lsts, data, wgts=None, lstbins=100, wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))):
    lstgrid = np.linspace(0, 2*np.pi, lstbins).astype(np.float32)
    if wgts is None: wgts = np.where(np.abs(data) == 0, 0, 1.).astype(np.float32)
    datwgt = g(lsts, lstgrid, np.vstack([data, wgts]))
    datgrid = datwgt[:lstbins]
    wgtgrid = datwgt[lstbins:]
    
    print('f', datwgt)
    lstgrid = np.linspace(0, 2*np.pi, lstbins).astype(np.float32)
    return lstgrid, datgrid, wgtgrid

@guvectorize([(float32[:], float32[:],float32[:,:], float32[:,:])], '(n),(n),(p,m)->(p,m)', target='cuda')
def g(lsts, lstgrid, datwgts, out):
    lstbins=100
    lstres = lstgrid[1]-lstgrid[0]
    data = datwgts[:lstbins]
    wgts = datwgts[lstbins:]
    sumgrid,wgtgrid = 0, 0
    #wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))
    for lst,d,w in zip(lsts,data,wgts):
        dt = lstgrid - lst
        #wf = wgtfunc(dt,lstres); wf.shape = (-1,) + (1,)*(data.ndim-1)
        wf = np.exp(-dt**2/(2*lstres**2)); wf.shape = (-1,) + (1,)*(data.ndim-1)
        d.shape, w.shape = (1,-1), (1,-1)
        #print lst, dt
        wgtgrid += w * wf
        sumgrid += d * w * wf 
    datgrid = np.where(wgtgrid > 1e-10, sumgrid/wgtgrid, 0)
    out = np.vstack([datgrid, wgtgrid])
    print('?',out[:lstbins])
    print(lsts.shape, datwgts.shape, out.shape)
    #lstsg, datag = lsts, data

In [ ]:
lsts = np.linspace(0, 6.28, 100, dtype=np.float32)
data = np.random.rand(100, 51).astype(np.float32)

In [ ]:
out = np.zeros_like(data)
#a,b,c = glst_grid(lsts, data,lstbins=100)
%timeit aa,bb,cc = lst_grid(lsts, data,lstbins=100)

In [ ]:
%%file basic.py
import numpy as np
def lst_grid(lsts, data, wgts=None, lstbins=6300, wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))):
    lstgrid = np.linspace(0, 2*np.pi, lstbins)
    lstres = lstgrid[1]-lstgrid[0]
    if wgts is None: wgts = np.where(np.abs(data) == 0, 0, 1.)
    sumgrid,wgtgrid = 0, 0
    for lst,d,w in zip(lsts,data,wgts):
        dt = lstgrid - lst
        wf = wgtfunc(dt,lstres); wf.shape = (-1,) + (1,)*(data.ndim-1)
        d.shape, w.shape = (1,-1), (1,-1)
        #print lst, dt
        wgtgrid += w * wf
        sumgrid += d * w * wf 
    datgrid = np.where(wgtgrid > 1e-10, sumgrid/wgtgrid, 0)
    #print('real', datgrid)
    return lstgrid, datgrid, wgtgrid
lsts = np.linspace(0, 6.28, 100, dtype=np.float32)
data = np.random.rand(100, 51).astype(np.float32)
def execute():
    aa,bb,cc = lst_grid(lsts, data,lstbins=100)

In [ ]:
import basic
%lprun -f basic.lst_grid basic.execute()

In [7]:
import numpy as np
from numba import guvectorize, cuda

@guvectorize(['void(float64[:], intp[:], float64[:])'], '(n),(m)->(n)', target='cuda')
def move_mean(a, window_arr, out):
    window_width = window_arr[0]
    asum = 0.0
    count = 0
    for i in range(window_width):
        asum += a[i]
        count += 1
        out[i] = asum / count
    for i in range(window_width, len(a)):
        asum += a[i] - a[i - window_width]
        out[i] = asum / count

arr = np.arange(2000, dtype=np.float64).reshape(200, 10)
#print arr 
%timeit move_mean(arr,np.array([10]))

/home/yunfanz/anaconda3/envs/envPAPER/lib/python2.7/site-packages/numba/cuda/compiler.py:233: UserWarning: Could not autotune, using default tpb of 128
  warnings.warn('Could not autotune, using default tpb of 128')


1000 loops, best of 3: 1.4 ms per loop


In [ ]:
from __future__ import print_function

import sys

import numpy as np

from numba import guvectorize, cuda

if sys.version_info[0] == 2:
    range = xrange

# Controls whether to manually handle CUDA memory allocation or not.
MANAGE_CUDA_MEMORY = True

#    function type:
#        - has no void return type
#        - array argument is one dimenion fewer than the source array
#        - scalar output is passed as a 1-element array.
#
#    signature: (n)->()
#        - the function takes an array of n-element and output a scalar.

@guvectorize(['void(int32[:], int32[:])'], '(n)->()', target='cuda')
def sum_row(inp, out):
    tmp = 0.
    for i in range(inp.shape[0]):
        tmp += inp[i]
    out[0] = tmp

# inp is (10000, 3)
# out is (10000)
# The outter (leftmost) dimension must match or numpy broadcasting is performed.
# But, broadcasting on CUDA arrays is not supported.

inp = np.arange(30000, dtype=np.int32).reshape(10000, 3)


if MANAGE_CUDA_MEMORY:
    # invoke on CUDA with manually managed memory
    out = np.empty(10000, dtype=inp.dtype)

    dev_inp = cuda.to_device(inp)             # alloc and copy input data
    dev_out = cuda.to_device(out, copy=False) # alloc only

    sum_row(dev_inp, out=dev_out)             # invoke the gufunc

    dev_out.copy_to_host(out)                 # retrieve the result
else:
    # Manually managing the CUDA allocation is optional, but recommended
    # for maximum performance.
    out = sum_row(inp)

# verify result
goal = np.empty_like(out)
for i in range(inp.shape[0]):
    assert out[i] == inp[i].sum()

# print out
print('input'.center(80, '-'))
print(inp)
print('output'.center(80, '-'))
print(out)

In [23]:
from numba import guvectorize
from numpy import arange

@guvectorize(['void(float32[:,:], float32[:,:], float32[:,:])'],
             '(m,n),(n,p)->(m,p)', target='parallel')
def matmul(A, B, C):
    m, n = A.shape
    n, p = B.shape
    for i in range(m):
        for j in range(p):
            C[i, j] = 0
            for k in range(n):
                C[i, j] += A[i, k] * B[k, j]

w = 2000
A = arange(w**2).reshape(w, w).astype(np.float32)
B = arange(w**2).reshape(w, w).astype(np.float32)
%timeit C = matmul(A, B)
%timeit C = np.dot(A,B)

1 loop, best of 3: 12.6 s per loop
10 loops, best of 3: 40.7 ms per loop


In [4]:
from numba import jit, vectorize, guvectorize, float64, complex64, int32, float32

@jit(int32(complex64, int32))
def mandelbrot(c,maxiter):
    nreal = 0
    real = 0
    imag = 0
    for n in range(maxiter):
        nreal = real*real - imag*imag + c.real
        imag = 2* real*imag + c.imag
        real = nreal;
        if real * real + imag * imag > 4.0:
            return n
    return 0

@guvectorize([(complex64[:], int32[:], int32[:])], '(n),()->(n)',target='parallel')
def mandelbrot_numpy(c, maxit, output):
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        output[i] = mandelbrot(c[i],maxiter)
        
def mandelbrot_set(xmin,xmax,ymin,ymax,width,height,maxiter):
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    n3 = mandelbrot_numpy(c,maxiter)
    return (r1,r2,n3.T) 
%timeit mandelbrot_set(-0.74877,-0.74872,0.06505,0.06510,1000,1000,2048)

1 loop, best of 3: 1.13 s per loop


In [5]:
@guvectorize([(complex64[:], int32[:], int32[:])], '(n),(n)->(n)', target='cuda')
def mandelbrot_numpy(c, maxit, output):
    maxiter = maxit[0]
    for i in range(c.shape[0]):
        creal = c[i].real
        cimag = c[i].imag
        real = creal
        imag = cimag
        output[i] = 0
        for n in range(maxiter):
            real2 = real*real
            imag2 = imag*imag
            if real2 + imag2 > 4.0:
                output[i] = n
                break
            imag = 2* real*imag + cimag
            real = real2 - imag2 + creal
            
        
def mandelbrot_set(xmin,xmax,ymin,ymax,width,height,maxiter):
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
    n3 = np.empty(c.shape, int)
    maxit = np.ones(c.shape, int) * maxiter
    n3 = mandelbrot_numpy(c,maxit)
    return (r1,r2,n3.T) 
%timeit mandelbrot_set(-0.74877,-0.74872,0.06505,0.06510,1000,1000,2048)

1 loop, best of 3: 533 ms per loop


In [6]:
import pycuda.driver as drv
import pycuda.tools
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
from pycuda.elementwise import ElementwiseKernel
complex_gpu = ElementwiseKernel(
    "pycuda::complex<float> *q, int *output, int maxiter",
    """
    {
        float nreal, real = 0;
        float imag = 0;
        output[i] = 0;
        for(int curiter = 0; curiter < maxiter; curiter++) {
            float real2 = real*real;
            float imag2 = imag*imag;
            nreal = real2 - imag2 + q[i].real();
            imag = 2* real*imag + q[i].imag();
            real = nreal;
            if (real2 + imag2 > 4.0f){
                output[i] = curiter;
                break;
                };
        };
    }
    """,
    "complex5",
    preamble="#include <pycuda-complex.hpp>",)

def mandelbrot_gpu(c, maxiter):
    q_gpu = gpuarray.to_gpu(c.astype(np.complex64))
    iterations_gpu = gpuarray.to_gpu(np.empty(c.shape, dtype=np.int))
    complex_gpu(q_gpu, iterations_gpu, maxiter)

    return iterations_gpu.get()
def mandelbrot_set(xmin,xmax,ymin,ymax,width,height,maxiter):
    r1 = np.linspace(xmin, xmax, width, dtype=np.float32)
    r2 = np.linspace(ymin, ymax, height, dtype=np.float32)
    c = r1 + r2[:,None]*1j
#     n3 = np.empty(c.shape, int)
#     maxit = np.ones(c.shape, int) * maxiter
    n3 = mandelbrot_gpu(c,maxiter)
    return (r1,r2,n3.T) 
%timeit mandelbrot_set(-0.74877,-0.74872,0.06505,0.06510,1000,1000,2048)

10 loops, best of 3: 45.5 ms per loop


In [ ]:
import pycuda.driver as drv
import pycuda.tools
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.gpuarray as gpuarray
from pycuda.elementwise import ElementwiseKernel
mod = SourceModule("""
    __global__ void step("float *lsts, float *lstg, pycuda::complex<float> *datas, float *wgts, pycuda::complex<float> *wgtg, pycuda::complex<float> *sumg")
    {
        float lstres = lstg[1]-lstg[0]
        int bx = blockInd.x
        
        output[i] = 0;
        for(int curiter = 0; curiter < maxiter; curiter++) {
            float real2 = real*real;
            float imag2 = imag*imag;
            nreal = real2 - imag2 + q[i].real();
            imag = 2* real*imag + q[i].imag();
            real = nreal;
            if (real2 + imag2 > 4.0f){
                output[i] = curiter;
                break;
                };
        };
    }
    """,
    preamble="#include <pycuda-complex.hpp>",)

def _pass_gpu(lsts, lstgrid, data, wgts, wgtfunc):
    lsts_gpu = gpuarray.to_gpu(lsts.astype(np.float32))
    lstgrid_gpu = gpuarray.to_gpu(lstgrid.astype(np.float32))
    data_gpu = gpuarray.to_gpu(data.astype(np.complex64))
    wgts_gpu = gpuarray.to_gpu(wgts.astype(np.float32))
    wgtfunc_gpu = gpuarray.to_gpu(wgtfunc.astype(np.float32))
    wgtgrid_gpu = gpuarray.to_gpu(np.empty((lstgrid.shape[0], data.shape[1]), dtype=np.float32))
    sumgrid_gpu = gpuarray.to_gpu(np.empty((lstgrid.shape[0], data.shape[1]), dtype=np.complex64))
    
def lst_grid_pycuda(lsts, data, wgts=None, lstbins=6300, wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))):
    lstgrid = np.linspace(0, 2*np.pi, lstbins)
    lstres = lstgrid[1]-lstgrid[0]
    if wgts is None: wgts = np.where(np.abs(data) == 0, 0, 1.)
    dt = lstgrid.reshape((1,-1))-lst.reshape((-1,1))
    wf = wgtfunc(dt,lstres)
    sumgrid,wgtgrid = 0, 0
    for lst,d,w in zip(lsts,data,wgts):
        dt = lstgrid - lst
        wf = wgtfunc(dt,lstres); wf.shape = (-1,) + (1,)*(data.ndim-1)
        d.shape, w.shape = (1,-1), (1,-1)
        #print lst, dt
        wgtgrid += w * wf
        sumgrid += d * w * wf 
    datgrid = np.where(wgtgrid > 1e-10, sumgrid/wgtgrid, 0)
    #print('real', datgrid)
    return lstgrid, datgrid, wgtgrid
@guvectorize([(float32[:], float32[:,:],float32[:,:], float32[:,:])], '(n),(n,m)->(n,m),(n,m)')
def g_cpu(lsts, data, lstsg, datag):
    lstsg, datag, _ = lst_grid(lsts, data)
    #lstsg, datag = lsts, data

In [40]:
from numba import float32, jit, vectorize, guvectorize
wgtfunc=lambda dt,res: np.exp(-dt**2/(2*res**2))
@jit('void(float32[:,:], float32)')
def jit_wgtfunc(dt, res):
    for ddt in dt:
        temp= np.exp(-ddt**2/(2*res**2))
@guvectorize([(float32, float32)], '(n,m), (p)')
def vec_wgtfunc(dt, res):
    for ddt in dt:
        temp= np.exp(-ddt**2/(2*res**2))
lsts = np.linspace(0,6.28,1000000).reshape((1000, 1000)).astype(np.float32)
dlst = np.float32(0.001)
%timeit jit_wgtfunc(lsts, dlst)
%timeit vec_wgtfunc(lsts, np.array([dlst]))

TypeError: scalar type float32 given for non scalar argument #1

0.001